In [67]:
import torch
import numpy as np
from PIL import Image

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [68]:
# Load YOLOv5 model
device = get_default_device()
model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True)
model.to(device)
model.eval()

PRO TIP 💡 Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 5.27M/5.27M [00:00<00:00, 47.1MB/s]


In [69]:
img = "bus.jpg"  # Test image
image = Image.open(img)
orig_width, orig_height = image.size
results = model(img)


image 1/1 c:\Users\danil\Desktop\Naturalistic-Adversarial-Patch\bus.jpg: 640x480 4 persons, 1 bus, 50.5ms
Speed: 6.0ms preprocess, 50.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


In [70]:
print(results)

[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant'

In [71]:
cls_id_attacked = 0 # person

In [72]:
# Extract bounding boxes and labels
bboxes = results.xyxy[0]  # Bounding boxes
labels = []  # format:  (label, x_center, y_center, w, h)
labels_rescale = []  # format:  (label, confidence, left, top, right, bottom)

print(bboxes)
for *xyxy, conf, cls in bboxes:
    detected_class = int(cls.cpu().item())

    if detected_class == cls_id_attacked:
        x_center = (xyxy[0] + xyxy[2]) / 2 / orig_width
        y_center = (xyxy[1] + xyxy[3]) / 2 / orig_height
        bbox_w = (xyxy[2] - xyxy[0]) / orig_width
        bbox_h = (xyxy[3] - xyxy[1]) / orig_height
        label = [cls.item(), x_center.item(), y_center.item(), bbox_w.item(), bbox_h.item(), conf.item()]
        labels.append(label)
        label_rescale = [cls.item(), conf.item(), xyxy[0].item(), xyxy[1].item(), xyxy[2].item(), xyxy[3].item()]
        labels_rescale.append(label_rescale)

labels = np.array(labels, dtype=np.float32)
labels_rescale = np.array(labels_rescale, dtype=np.float32)

print(f"Labels: {labels}")
print(f"Labels rescale: {labels_rescale}")


AttributeError: 'list' object has no attribute 'xyxy'

In [ ]:
import shutil
cache_dir = 'C:/Users/danil/.cache/torch/hub/ultralytics_yolov5_master'
shutil.rmtree(cache_dir, ignore_errors=True)
